In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error,r2_score,mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Application_II/practical_application_II_starter/data/gold/vehicles_gold.csv')
df.head()

,id,region,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,price
0,7222695916,prescott,2014.0,infiniti,g series,fair,5 cylinders,gas,96737.302119,clean,automatic,4wd,full-size,offroad,grey,6000
1,7218891961,fayetteville,2014.0,infiniti,g series,fair,5 cylinders,gas,96737.302119,clean,automatic,4wd,full-size,offroad,grey,11900
2,7221797935,florida keys,2014.0,infiniti,g series,fair,5 cylinders,gas,96737.302119,clean,automatic,4wd,full-size,offroad,grey,21000
3,7222270760,worcester / central MA,2014.0,infiniti,g series,fair,5 cylinders,gas,96737.302119,clean,automatic,4wd,full-size,offroad,grey,1500
4,7210384030,greensboro,2014.0,infiniti,g series,fair,5 cylinders,gas,96737.302119,clean,automatic,4wd,full-size,offroad,grey,4900


In [4]:
df2=df.copy()
df.drop(['id','title_status','model','region'],axis=1,inplace=True)
df.head()

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,price
0,2014.0,infiniti,fair,5 cylinders,gas,96737.302119,automatic,4wd,full-size,offroad,grey,6000
1,2014.0,infiniti,fair,5 cylinders,gas,96737.302119,automatic,4wd,full-size,offroad,grey,11900
2,2014.0,infiniti,fair,5 cylinders,gas,96737.302119,automatic,4wd,full-size,offroad,grey,21000
3,2014.0,infiniti,fair,5 cylinders,gas,96737.302119,automatic,4wd,full-size,offroad,grey,1500
4,2014.0,infiniti,fair,5 cylinders,gas,96737.302119,automatic,4wd,full-size,offroad,grey,4900


In [5]:
df2[df2.year==2012.0].head()

,id,region,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,price
26,7316343444,auburn,2012.0,gmc,sierra 2500 hd extended cab,good,8 cylinders,gas,68696.0,clean,other,4wd,full-size,pickup,black,27990
51,7313406529,auburn,2012.0,honda,odyssey,excellent,6 cylinders,gas,95000.0,clean,automatic,fwd,full-size,mini-van,silver,14000
62,7312342811,auburn,2012.0,toyota,tacoma access cab pickup,good,5 cylinders,gas,37725.0,clean,other,4wd,full-size,pickup,white,22990
72,7311327726,auburn,2012.0,toyota,tacoma double cab pickup,good,6 cylinders,gas,43182.0,clean,other,4wd,full-size,pickup,white,29990
126,7304685502,auburn,2012.0,jeep,wrangler unlimited sport,good,6 cylinders,gas,52172.0,clean,other,4wd,full-size,SUV,black,27590


In [6]:
num_col=['year','odometer']
cat_cols=['manufacturer','condition','cylinders','fuel','transmission','drive','size','type','paint_color']

In [7]:
temp=[]
for i in cat_cols:
   temp.append(df[i].unique().flatten()) 
   

In [8]:
temp

[array(['infiniti', 'gmc', 'chevrolet', 'toyota', 'ford', 'jeep', 'nissan',
        'ram', 'mazda', 'cadillac', 'honda', 'dodge', 'lexus', 'jaguar',
        'buick', 'chrysler', 'volvo', 'audi', 'lincoln', 'alfa-romeo',
        'subaru', 'acura', 'hyundai', 'mercedes-benz', 'bmw', 'mitsubishi',
        'volkswagen', 'porsche', 'kia', 'rover', 'ferrari', 'mini',
        'pontiac', 'fiat', 'tesla', 'saturn', 'mercury', 'harley-davidson',
        'aston-martin', 'land rover', 'morgan'], dtype=object),
 array(['fair', 'good', 'excellent', 'like new', 'new', 'salvage'],
       dtype=object),
 array(['5 cylinders', '8 cylinders', '6 cylinders', '4 cylinders',
        '3 cylinders', '10 cylinders', 'other', '12 cylinders'],
       dtype=object),
 array(['gas', 'other', 'diesel', 'hybrid', 'electric'], dtype=object),
 array(['automatic', 'other', 'manual'], dtype=object),
 array(['4wd', 'rwd', 'fwd'], dtype=object),
 array(['full-size', 'mid-size', 'compact', 'sub-compact'], dtype=object),
 ar

In [9]:
df[cat_cols]=df[cat_cols].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364420 entries, 0 to 364419
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   year          364420 non-null  float64 
 1   manufacturer  364420 non-null  category
 2   condition     364420 non-null  category
 3   cylinders     364420 non-null  category
 4   fuel          364420 non-null  category
 5   odometer      364420 non-null  float64 
 6   transmission  364420 non-null  category
 7   drive         364420 non-null  category
 8   size          364420 non-null  category
 9   type          364420 non-null  category
 10  paint_color   364420 non-null  category
 11  price         364420 non-null  int64   
dtypes: category(9), float64(2), int64(1)
memory usage: 11.5 MB


In [10]:
columnsValue={}
for col in cat_cols:
    column_labels=list(range(1,len(df[col].unique())+1))
    rep={}
    for i,j in zip(column_labels,df[col].unique()):
        rep[j]=i
    columnsValue[col]=list(df[col].unique())
    df[col].replace(rep,inplace=True)

In [11]:
df.head()

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,price
0,2014.0,1,1,1,1,96737.302119,1,1,1,1,1,6000
1,2014.0,1,1,1,1,96737.302119,1,1,1,1,1,11900
2,2014.0,1,1,1,1,96737.302119,1,1,1,1,1,21000
3,2014.0,1,1,1,1,96737.302119,1,1,1,1,1,1500
4,2014.0,1,1,1,1,96737.302119,1,1,1,1,1,4900


In [12]:
norm = StandardScaler()

norm.fit(df[['year','odometer']])
standardvalues=norm.transform(df[['year','odometer']])

df['year']=standardvalues[:,:1].flatten()
df['odometer']=standardvalues[:,1:].flatten()

pickle.dump(norm, open('/content/drive/MyDrive/Colab Notebooks/Application_II/practical_application_II_starter/models/StandardScaler.pkl', 'wb'))

df['price'] = np.log(df['price'])
#scaling target variable
q1,q3=(df['price'].quantile([0.25,0.75]))
o1=q1-1.5*(q3-q1)
o2=q3+1.5*(q3-q1)
df=df[(df.price>=o1) & (df.price<=o2)]

In [13]:
df.head()

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,price
0,0.271095,1,1,1,1,0.081454,1,1,1,1,1,8.699515
1,0.271095,1,1,1,1,0.081454,1,1,1,1,1,9.384294
2,0.271095,1,1,1,1,0.081454,1,1,1,1,1,9.952278
3,0.271095,1,1,1,1,0.081454,1,1,1,1,1,7.313220
4,0.271095,1,1,1,1,0.081454,1,1,1,1,1,8.496990


### XGBoost Evaluation

In [14]:
X=df.iloc[:,:-1]
y=df['price']

xg_reg2 = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.4,
                max_depth = 24, alpha = 5, n_estimators = 200)
xg_reg2.fit(X,y)
pickle.dump(xg_reg2, open('/content/drive/MyDrive/Colab Notebooks/Application_II/practical_application_II_starter/models/XGBoostDeploy.pkl', 'wb'))

In [15]:
standardscaler= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Application_II/practical_application_II_starter/models/StandardScaler.pkl', 'rb'))
mymodel = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Application_II/practical_application_II_starter/models/XGBoostDeploy.pkl', 'rb'))

In [16]:
df.sample(2)

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,price
321584,-0.457065,25,3,3,1,0.861129,1,1,1,9,2,8.852951
13434,-2.277465,9,1,4,1,0.123445,1,2,3,11,3,7.313220


In [17]:
df2.iloc[107333]

id                 7311592208
region           northwest GA
year                   2013.0
manufacturer    mercedes-benz
model                gl-class
condition                fair
cylinders         8 cylinders
fuel                      gas
odometer              88890.0
title_status            clean
transmission        automatic
drive                     4wd
size                 mid-size
type                      SUV
paint_color             brown
price                   20949
Name: 107333, dtype: object

In [18]:
year_odometer=pd.DataFrame(data=[[2020.0,6052]],columns=['year','odometer'])
x=standardscaler.transform(year_odometer[['year','odometer']]).flatten()

In [19]:
#get col uncoded value
df2.manufacturer.unique()[8-1]

'ram'

In [20]:
test=pd.DataFrame(data=[[x[0],8,3,1,1,x[1],1,3,1,2,1]],columns=X.columns)
test

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color
0,1.363335,8,3,1,1,-1.404454,1,3,1,2,1


In [21]:
# paint_color, manufacturer, year, transmission, cylinders, size, fuel, condition, drive, type, odometer

In [22]:
t=[1,8,x[0],1,1,1,1,3,3,2,x[1]]
t

[1, 8, 1.3633354436820357, 1, 1, 1, 1, 3, 3, 2, -1.404453996066564]

In [23]:
pred=mymodel.predict(test)
price=np.exp(pred[0])
print(price)

28034.143


### Random Forest evaluation

In [33]:
RFR = RandomForestRegressor(n_estimators=180,random_state=0, min_samples_leaf=1, max_features=0.5, n_jobs=-1, oob_score=True)
RFR.fit(X,y)
pickle.dump(RFR, open('/content/drive/MyDrive/Colab Notebooks/Application_II/practical_application_II_starter/models/RFRDeploy.pkl', 'wb'))

In [25]:
standardscaler= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Application_II/practical_application_II_starter/models/StandardScaler.pkl', 'rb'))
myRFRmodel = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Application_II/practical_application_II_starter/models/RFRDeploy.pkl', 'rb'))

In [26]:
df.sample(2)

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,price
316682,0.271095,8,3,3,3,-1.503602,3,1,1,2,2,10.491135
62774,0.635175,7,2,4,1,1.410594,1,3,1,9,1,8.986572


In [27]:
df2.iloc[107333]

id                 7311592208
region           northwest GA
year                   2013.0
manufacturer    mercedes-benz
model                gl-class
condition                fair
cylinders         8 cylinders
fuel                      gas
odometer              88890.0
title_status            clean
transmission        automatic
drive                     4wd
size                 mid-size
type                      SUV
paint_color             brown
price                   20949
Name: 107333, dtype: object

In [28]:
year_odometer=pd.DataFrame(data=[[2020.0,6052]],columns=['year','odometer'])

In [29]:
x=standardscaler.transform(year_odometer[['year','odometer']]).flatten()
df2.manufacturer.unique()[8-1]

'ram'

In [30]:
test=pd.DataFrame(data=[[x[0],8,3,1,1,x[1],1,3,1,2,1]],columns=X.columns)
test

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color
0,1.363335,8,3,1,1,-1.404454,1,3,1,2,1


In [31]:
t=[1,8,x[0],1,1,1,1,3,3,2,x[1]]
t

[1, 8, 1.3633354436820357, 1, 1, 1, 1, 3, 3, 2, -1.404453996066564]

In [32]:
pred=myRFRmodel.predict(test)
price=np.exp(pred[0])
print(price)

34395.31971582947
